# VAZHI Dataset Factory v4.0

Constructs the curated SFT dataset from all source data per [ADR-010](../docs/adr/010-data-pipeline-architecture.md).

**Pipeline:** Source Data (local) → Dataset Factory (this notebook) → Training → Artifacts (HuggingFace)

**Key features:**
1. Loads vazhi-packs (6 domain packs, 3,007 Q&A pairs)
2. Streams IndicAlign diversity samples (~1,500 Tamil samples)
3. Includes handcrafted samples (refusal, brevity, greeting, guardrails)
4. Loads general knowledge Q&A from LEGACY files
5. Anti-memorization filter for Thirukkural verbatim Q&As
6. **Hard-enforces** composition targets (fails if violated)
7. Stratified train/eval split (90/10) by source bucket
8. Uploads to HuggingFace with version tracking

**Run on:** Kaggle (P100) or Colab (T4)

**Output:** `CryptoYogi/vazhi-tamil-sft-v4_0` with `train` and `validation` splits

## 1. Config & Dependencies

In [ ]:
!pip install -q datasets huggingface_hub

import json
import os
import re
import random
from collections import Counter
from pathlib import Path

from datasets import load_dataset, Dataset, DatasetDict
from huggingface_hub import login, HfApi

# === CONFIG ===
VERSION = "4.0"
OUTPUT_DATASET = "CryptoYogi/vazhi-tamil-sft-v4_0"
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

SYSTEM_PROMPT = (
    "\u0ba8\u0bc0\u0b99\u0bcd\u0b95\u0bb3\u0bcd VAZHI (\u0bb5\u0bb4\u0bbf), \u0ba4\u0bae\u0bbf\u0bb4\u0bcd \u0bae\u0b95\u0bcd\u0b95\u0bb3\u0bc1\u0b95\u0bcd\u0b95\u0bbe\u0ba9 AI \u0b89\u0ba4\u0bb5\u0bbf\u0baf\u0bbe\u0bb3\u0bb0\u0bcd. "
    "\u0ba4\u0bae\u0bbf\u0bb4\u0bbf\u0bb2\u0bcd \u0ba4\u0bc6\u0bb3\u0bbf\u0bb5\u0bbe\u0b95\u0bb5\u0bc1\u0bae\u0bcd \u0b89\u0ba4\u0bb5\u0bbf\u0baf\u0bbe\u0b95\u0bb5\u0bc1\u0bae\u0bcd \u0baa\u0ba4\u0bbf\u0bb2\u0bb3\u0bbf\u0baf\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd. "
    '\u0ba4\u0bc6\u0bb0\u0bbf\u0baf\u0bbe\u0bb5\u0bbf\u0b9f\u0bcd\u0b9f\u0bbe\u0bb2\u0bcd "\u0ba4\u0bc6\u0bb0\u0bbf\u0baf\u0bb5\u0bbf\u0bb2\u0bcd\u0bb2\u0bc8" \u0b8e\u0ba9\u0bcd\u0bb1\u0bc1 \u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd.'
)

# Hard composition constraints (Factory fails if violated)
COMPOSITION_TARGETS = {
    "domain_packs": {"target": 0.45, "min": 0.40, "max": 0.50},
    "indicalign":   {"target": 0.30, "min": 0.25, "max": 0.35},
    "kural_interp": {"target": 0.15, "min": 0.00, "max": 0.15},
    "handcrafted":  {"target": 0.03, "min": 0.02, "max": 0.05},
    "general":      {"target": 0.07, "min": 0.05, "max": 0.10},
}

MAX_CHAR_LENGTH = 1500
MIN_TAMIL_PCT = 30  # Minimum Tamil character percentage

print(f"\u2705 Config loaded: Dataset Factory v{VERSION}")
print(f"   Output: {OUTPUT_DATASET}")
print(f"   Composition targets:")
for name, cfg in COMPOSITION_TARGETS.items():
    print(f"     {name}: {cfg['target']:.0%} (min {cfg['min']:.0%}, max {cfg['max']:.0%})")

In [ ]:
# Login to HuggingFace
# On Kaggle:
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    hf_token = secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("\u2705 Logged in via Kaggle secrets")
except Exception:
    # On Colab or local: use HF_TOKEN env var or interactive login
    try:
        from google.colab import userdata
        hf_token = userdata.get('HF_TOKEN')
        login(token=hf_token)
        print("\u2705 Logged in via Colab secrets")
    except Exception:
        login()
        print("\u2705 Logged in interactively")

## 2. Helper Functions

In [ ]:
def to_chatml(instruction, output, system_prompt=None):
    """Convert instruction/output pair to strict ChatML format."""
    sp = system_prompt or SYSTEM_PROMPT
    return (
        f"<|im_start|>system\n{sp}<|im_end|>\n"
        f"<|im_start|>user\n{instruction}<|im_end|>\n"
        f"<|im_start|>assistant\n{output}<|im_end|>"
    )


CHATML_PATTERN = re.compile(
    r'<\|im_start\|>system\n.+?<\|im_end\|>\n'
    r'<\|im_start\|>user\n(.+?)<\|im_end\|>\n'
    r'<\|im_start\|>assistant\n(.+?)<\|im_end\|>',
    re.DOTALL
)


def validate_chatml_strict(text):
    """Validate a sample has proper ChatML with non-empty user AND assistant."""
    match = CHATML_PATTERN.search(text)
    if not match:
        return False, "no ChatML structure found"
    user_content = match.group(1).strip()
    assistant_content = match.group(2).strip()
    if len(user_content) < 2:
        return False, "empty user content"
    if len(assistant_content) < 2:
        return False, "empty assistant content"
    return True, "ok"


def count_tamil_chars(text):
    """Count Tamil Unicode characters."""
    return sum(1 for c in text if '\u0b80' <= c <= '\u0bff')


def tamil_char_pct(text):
    """Get Tamil character percentage."""
    if not text:
        return 0.0
    return 100.0 * count_tamil_chars(text) / len(text)


def is_verbatim_kural_qa(question, answer):
    """Reject Q&As that ask for exact verse text. Err on the side of rejecting."""
    verbatim_patterns = [
        r'\u0b95\u0bc1\u0bb1\u0bb3\u0bcd\s*\d+\s*(\u0b8e\u0ba9\u0bcd\u0ba9|\u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1|\u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd|\u0b8e\u0bb4\u0bc1\u0ba4\u0bbf\s*\u0b95\u0bbe\u0b9f\u0bcd\u0b9f\u0bc1|\u0b95\u0bc2\u0bb1\u0bc1\u0b95)',
        r'(first|\u0bae\u0bc1\u0ba4\u0bb2\u0bcd)\s*\u0b95\u0bc1\u0bb1\u0bb3\u0bcd\s*(\u0b8e\u0ba9\u0bcd\u0ba9|\u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1|\u0b9a\u0bca\u0bb2\u0bcd\u0bb2\u0bc1\u0b99\u0bcd\u0b95\u0bb3\u0bcd)',
        r'\u0ba4\u0bbf\u0bb0\u0bc1\u0b95\u0bcd\u0b95\u0bc1\u0bb1\u0bb3\u0bbf\u0ba9\u0bcd\s+\u0bae\u0bc1\u0ba4\u0bb2\u0bcd\s+\u0b95\u0bc1\u0bb1\u0bb3\u0bcd',
        r'\u0b95\u0bc1\u0bb1\u0bb3\u0bcd\s*[\d]+(?:\s*\u0b90)?\s*\u0b8e\u0bb4\u0bc1\u0ba4\u0bbf',
    ]
    for pat in verbatim_patterns:
        if re.search(pat, question, re.IGNORECASE):
            return True
    # Also reject if answer looks like a raw couplet (short + has newline = verse format)
    if len(answer) < 200 and "\n" in answer and not any(
        w in answer for w in [
            "\u0bb5\u0bbf\u0bb3\u0b95\u0bcd\u0b95\u0bae\u0bcd",
            "\u0baa\u0bca\u0bb0\u0bc1\u0bb3\u0bcd",
            "\u0b85\u0bb0\u0bcd\u0ba4\u0bcd\u0ba4\u0bae\u0bcd",
        ]
    ):
        return True
    return False


# Self-test
good = to_chatml("test question", "test answer")
valid, reason = validate_chatml_strict(good)
assert valid, f"Self-test failed: {reason}"
print("\u2705 Helper functions defined and self-tested")

## 3. Load Domain Packs (vazhi-packs)

In [ ]:
# Load flattened vazhi-packs from data/sources/sft/vazhi-packs/
# On Kaggle: upload the data/sources/ directory as a dataset or clone the repo

PACKS_DIR = Path("../data/sources/sft/vazhi-packs")
if not PACKS_DIR.exists():
    # Try Kaggle input path
    PACKS_DIR = Path("/kaggle/input/vazhi-sources/sft/vazhi-packs")
if not PACKS_DIR.exists():
    raise FileNotFoundError(
        "Cannot find vazhi-packs. Upload data/sources/ as a Kaggle dataset "
        "or clone the repo."
    )

domain_pack_samples = []
kural_interp_samples = []

for pack_file in sorted(PACKS_DIR.glob("*.json")):
    pack_name = pack_file.stem
    with open(pack_file, encoding="utf-8") as f:
        pairs = json.load(f)
    
    pack_domain = []
    pack_kural = []
    pack_verbatim_rejected = 0
    
    for pair in pairs:
        instruction = pair["instruction"]
        output = pair["output"]
        
        # Length filter
        if len(instruction) + len(output) > MAX_CHAR_LENGTH:
            continue
        
        # Check if this is a Kural-related Q&A
        is_kural = any(
            k in instruction + output
            for k in ["\u0b95\u0bc1\u0bb1\u0bb3\u0bcd", "\u0ba4\u0bbf\u0bb0\u0bc1\u0b95\u0bcd\u0b95\u0bc1\u0bb1\u0bb3\u0bcd", "\u0ba4\u0bbf\u0bb0\u0bc1\u0bb5\u0bb3\u0bcd\u0bb3\u0bc1\u0bb5\u0bb0\u0bcd"]
        )
        
        if is_kural:
            # Apply anti-memorization filter
            if is_verbatim_kural_qa(instruction, output):
                pack_verbatim_rejected += 1
                continue
            pack_kural.append((instruction, output, pair.get("category", "")))
        else:
            pack_domain.append((instruction, output, pair.get("category", "")))
    
    domain_pack_samples.extend(pack_domain)
    kural_interp_samples.extend(pack_kural)
    
    print(f"  {pack_name}: {len(pack_domain)} domain, {len(pack_kural)} kural interpretive, {pack_verbatim_rejected} verbatim rejected")

print(f"\n\u2705 Domain packs: {len(domain_pack_samples)} samples")
print(f"\u2705 Kural interpretive: {len(kural_interp_samples)} samples")

## 4. Load IndicAlign Diversity Samples

In [ ]:
# Stream IndicAlign Tamil samples from HuggingFace
# Subsets: Dolly_T, WikiHow, Wiki_Conv, OpenAssistant_T

INDICALIGN_SUBSETS = ["Dolly_T", "WikiHow", "Wiki_Conv", "OpenAssistant_T"]
TARGET_INDICALIGN = 1500  # Total samples to collect
PER_SUBSET_TARGET = TARGET_INDICALIGN // len(INDICALIGN_SUBSETS)

indicalign_samples = []

for subset in INDICALIGN_SUBSETS:
    try:
        ds = load_dataset(
            "ai4bharat/indic-align",
            subset,
            split="train",
            streaming=True,
        )
        
        subset_samples = []
        for item in ds:
            if len(subset_samples) >= PER_SUBSET_TARGET:
                break
            
            # IndicAlign has 'messages' field with role/content pairs
            messages = item.get("messages", [])
            if not messages:
                # Try alternate format
                instruction = item.get("instruction", item.get("input", ""))
                output = item.get("output", item.get("response", ""))
            else:
                # Extract user and assistant messages
                instruction = ""
                output = ""
                for msg in messages:
                    role = msg.get("role", "")
                    content = msg.get("content", "")
                    if role == "user" and not instruction:
                        instruction = content
                    elif role == "assistant" and not output:
                        output = content
            
            if not instruction or not output:
                continue
            
            # Length and Tamil quality filters
            combined = instruction + output
            if len(combined) > MAX_CHAR_LENGTH:
                continue
            if tamil_char_pct(combined) < MIN_TAMIL_PCT:
                continue
            
            subset_samples.append((instruction, output, f"indicalign_{subset.lower()}"))
        
        indicalign_samples.extend(subset_samples)
        print(f"  {subset}: {len(subset_samples)} samples")
    
    except Exception as e:
        print(f"  {subset}: FAILED - {e}")

print(f"\n\u2705 IndicAlign total: {len(indicalign_samples)} samples")

## 5. Load Handcrafted Samples

In [ ]:
HANDCRAFTED_DIR = Path("../data/sources/sft/handcrafted")
if not HANDCRAFTED_DIR.exists():
    HANDCRAFTED_DIR = Path("/kaggle/input/vazhi-sources/sft/handcrafted")
if not HANDCRAFTED_DIR.exists():
    raise FileNotFoundError("Cannot find handcrafted samples directory.")

handcrafted_samples = []

for hc_file in sorted(HANDCRAFTED_DIR.glob("*.json")):
    with open(hc_file, encoding="utf-8") as f:
        items = json.load(f)
    
    for item in items:
        handcrafted_samples.append((
            item["instruction"],
            item["output"],
            item.get("category", hc_file.stem),
        ))
    
    print(f"  {hc_file.stem}: {len(items)} samples")

print(f"\n\u2705 Handcrafted total: {len(handcrafted_samples)} samples")

## 6. Load General Knowledge (LEGACY Q&A)

In [ ]:
# Select ~300 general knowledge Q&As from LEGACY files
# These cover dialects, emotions, daily routines, weather, etc.

LEGACY_DIR = Path("../data/LEGACY")
if not LEGACY_DIR.exists():
    LEGACY_DIR = Path("/kaggle/input/vazhi-legacy")

GENERAL_FILES = [
    "06_health.json",
    "09_weather.json",
    "10_shopping.json",
    "12_daily_routines.json",
    "13_emotions.json",
    "14_chennai_dialect.json",
    "15_madurai_dialect.json",
    "16_kongu_dialect.json",
    "31_malaysia_dialect.json",
    "03_numbers_time.json",
]

TARGET_GENERAL = 350
general_pool = []

for fname in GENERAL_FILES:
    fpath = LEGACY_DIR / fname
    if not fpath.exists():
        print(f"  {fname}: NOT FOUND, skipping")
        continue
    
    with open(fpath, encoding="utf-8") as f:
        items = json.load(f)
    
    file_samples = []
    for item in items:
        instruction = item.get("instruction", item.get("question", ""))
        output = item.get("output", item.get("answer", ""))
        if not instruction or not output:
            continue
        if len(instruction) + len(output) > MAX_CHAR_LENGTH:
            continue
        category = item.get("category", fname.replace(".json", ""))
        file_samples.append((instruction, output, category))
    
    general_pool.extend(file_samples)
    print(f"  {fname}: {len(file_samples)} usable samples")

# Sample down to target
if len(general_pool) > TARGET_GENERAL:
    general_samples = random.sample(general_pool, TARGET_GENERAL)
else:
    general_samples = general_pool

print(f"\n\u2705 General knowledge: {len(general_samples)} samples (from {len(general_pool)} pool)")

## 7. Compose, Validate & Enforce Composition

In [ ]:
def enforce_composition(buckets, targets):
    """Sample each bucket to target size. FAIL if any bucket is short of minimum."""
    total_target = sum(len(v) for v in buckets.values())
    # Use domain_packs size to anchor the total
    anchor_size = len(buckets["domain_packs"])
    estimated_total = int(anchor_size / targets["domain_packs"]["target"])
    
    result = {}
    for name, cfg in targets.items():
        available = buckets.get(name, [])
        target_count = int(estimated_total * cfg["target"])
        min_count = int(estimated_total * cfg["min"])
        max_count = int(estimated_total * cfg["max"])
        
        if len(available) < min_count:
            raise ValueError(
                f"FACTORY FAILED: '{name}' has {len(available)} samples, "
                f"minimum required is {min_count} ({cfg['min']:.0%} of ~{estimated_total}). "
                f"Add more data to this bucket before retrying."
            )
        
        # Sample to target, respecting max
        use_count = min(len(available), target_count, max_count)
        use_count = max(use_count, min_count)  # At least minimum
        
        if len(available) > use_count:
            result[name] = random.sample(available, use_count)
        else:
            result[name] = available
        
        pct = len(result[name]) / estimated_total * 100
        print(f"  {name}: {len(result[name])}/{len(available)} available ({pct:.1f}%)")
    
    return result


# Build buckets
buckets = {
    "domain_packs": domain_pack_samples,
    "indicalign": indicalign_samples,
    "kural_interp": kural_interp_samples,
    "handcrafted": handcrafted_samples,
    "general": general_samples,
}

print("\U0001f3af Enforcing composition targets...")
print(f"   Raw bucket sizes:")
for name, samples in buckets.items():
    print(f"     {name}: {len(samples)}")

composed = enforce_composition(buckets, COMPOSITION_TARGETS)

print(f"\n\u2705 Composition enforcement passed!")

In [ ]:
# Convert all to ChatML and validate
all_samples = []
seen_instructions = set()  # Dedup by instruction prefix

for bucket_name, samples in composed.items():
    for instruction, output, category in samples:
        # Dedup by first 100 chars of instruction
        dedup_key = instruction[:100]
        if dedup_key in seen_instructions:
            continue
        seen_instructions.add(dedup_key)
        
        # Convert to ChatML
        text = to_chatml(instruction, output)
        
        # Validate ChatML
        valid, reason = validate_chatml_strict(text)
        if not valid:
            print(f"  REJECTED ({bucket_name}): {reason} - {instruction[:50]}")
            continue
        
        # Tamil percentage check (skip for handcrafted brevity/greeting)
        if bucket_name not in ["handcrafted"] and tamil_char_pct(text) < MIN_TAMIL_PCT:
            continue
        
        all_samples.append({
            "text": text,
            "bucket": bucket_name,
            "category": category,
        })

# Shuffle
random.shuffle(all_samples)

print(f"\n\U0001f4ca Final dataset: {len(all_samples)} samples")
bucket_counts = Counter(s["bucket"] for s in all_samples)
for bucket, count in sorted(bucket_counts.items()):
    pct = 100 * count / len(all_samples)
    print(f"  {bucket}: {count} ({pct:.1f}%)")

## 8. Stratified Train/Eval Split

In [ ]:
# 90/10 stratified split by bucket
EVAL_RATIO = 0.10

train_samples = []
eval_samples = []

# Group by bucket
by_bucket = {}
for s in all_samples:
    bucket = s["bucket"]
    if bucket not in by_bucket:
        by_bucket[bucket] = []
    by_bucket[bucket].append(s)

for bucket, samples in by_bucket.items():
    random.shuffle(samples)
    n_eval = max(1, int(len(samples) * EVAL_RATIO))
    eval_samples.extend(samples[:n_eval])
    train_samples.extend(samples[n_eval:])

random.shuffle(train_samples)
random.shuffle(eval_samples)

print(f"\U0001f4ca Stratified split:")
print(f"  Train: {len(train_samples)}")
print(f"  Eval:  {len(eval_samples)}")
print(f"  Eval ratio: {len(eval_samples) / (len(train_samples) + len(eval_samples)):.1%}")

# Verify eval has all buckets
eval_buckets = Counter(s["bucket"] for s in eval_samples)
print(f"\n  Eval bucket distribution:")
for bucket, count in sorted(eval_buckets.items()):
    print(f"    {bucket}: {count}")

## 9. Upload to HuggingFace

In [ ]:
train_ds = Dataset.from_list(train_samples)
eval_ds = Dataset.from_list(eval_samples)

dataset_dict = DatasetDict({
    "train": train_ds,
    "validation": eval_ds,
})

api = HfApi()
api.create_repo(OUTPUT_DATASET, repo_type="dataset", exist_ok=True)

dataset_dict.push_to_hub(OUTPUT_DATASET)

print(f"\n\u2705 Dataset uploaded: https://huggingface.co/datasets/{OUTPUT_DATASET}")
print(f"   Train: {len(train_ds)} samples")
print(f"   Validation: {len(eval_ds)} samples")

## 10. Summary & Sample Outputs

In [ ]:
print("=" * 60)
print(f"VAZHI Dataset Factory v{VERSION} — Summary")
print("=" * 60)
print(f"\nTotal samples: {len(all_samples)}")
print(f"Train: {len(train_samples)} | Eval: {len(eval_samples)}")
print(f"\nComposition:")

total = len(all_samples)
for bucket, count in sorted(bucket_counts.items()):
    target = COMPOSITION_TARGETS[bucket]
    pct = count / total
    status = "\u2705" if target["min"] <= pct <= target["max"] else "\u274c"
    print(f"  {status} {bucket}: {count} ({pct:.1%}) [target: {target['target']:.0%}, range: {target['min']:.0%}-{target['max']:.0%}]")

# Show sample outputs from each bucket
print(f"\n{'=' * 60}")
print("Sample outputs (1 per bucket):")
print("=" * 60)

shown_buckets = set()
for s in all_samples:
    if s["bucket"] not in shown_buckets:
        shown_buckets.add(s["bucket"])
        print(f"\n[{s['bucket'].upper()}] category={s['category']}")
        # Extract user and assistant from ChatML
        match = CHATML_PATTERN.search(s["text"])
        if match:
            print(f"  Q: {match.group(1)[:100]}")
            print(f"  A: {match.group(2)[:150]}")
    if len(shown_buckets) == len(COMPOSITION_TARGETS):
        break

print(f"\n\u2705 Dataset Factory v{VERSION} complete!")
print(f"   Dataset: {OUTPUT_DATASET}")
print(f"   Next step: Use this dataset in training notebook")